In [20]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
#from sagemaker.predictor import csv_serializer 
from sagemaker.serializers import CSVSerializer

In [35]:
test_data = pd.read_csv('./score.csv')
label = test_data.iloc[:,:1]
test_data

,0,25,1,999,0.1,1.1,0.2,0.3,0.4,0.5,...,0.38,0.39,0.40,1.10,0.41,0.42,0.43,0.44,1.11,0.45
0,1,28,3,999,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1,38,1,999,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,32,1,999,1,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
3,0,40,1,999,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,0,55,3,999,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4113,0,29,3,999,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
4114,1,60,2,6,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4115,0,36,4,999,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4116,1,42,1,999,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [14]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-01-04-19-51-28-673
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-01-04-19-51-28-673
INFO:sagemaker:Creating endpoint with name xgboost-2024-01-04-19-51-28-673


-----!

In [37]:
xgb_predictor.serializer = CSVSerializer()
xgb_predictor.content_types = "text/csv"

xgb_predictor.predict(test_data.drop(['0'], axis=1).iloc[1,:])

/opt/conda/lib/python3.10/site-packages/sagemaker/base_serializers.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  has_multiple_rows = len(data) > 0 and self._is_sequence_like(data[0])


b'0.14644621312618256'

In [38]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [39]:
predictions = predict(test_data.drop(['0'], axis=1).values)

In [40]:
pd.crosstab(index=label['0'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0,3582,53
1,382,101


In [44]:
xgb_predictor.delete_model()
transformer.delete_model()
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

INFO:sagemaker:Deleting model with name: xgboost-2024-01-04-19-51-28-673


Exception: One or more models cannot be deleted, please retry. 
Failed models: xgboost-2024-01-04-19-51-28-673